# Import

In [1]:
import functools
import numpy as np
from scipy.linalg import expm

import qiskit as qk
import qiskit_dynamics as qk_d
import qiskit.providers.fake_provider as qk_fp

import qutip as qt
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import importlib

/home/andy/.cache/pypoetry/virtualenvs/pulse-simulator-0UjnMsVt-py3.10/lib/python3.10/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(


In [2]:
# configure jax to use 64 bit mode
import jax
jax.config.update("jax_enable_x64", True)

# tell JAX we are using CPU
jax.config.update('jax_platform_name', 'cpu')

# set default backend
qk_d.array.Array.set_default_backend('jax')
qk_d.array.Array.default_backend()

'jax'

In [3]:
import sys
sys.path.append("../")
import pulse_simulator as ps

# Inspect two qubit gate Hamiltonian

In [4]:
backend = qk_fp.FakeManila()

# Initialize device
# =====
# Undo units
units = 1e9
GHz = 1/units
ns = units

dt = backend.configuration().dt * ns  
duration = 220 * dt  # ns

registers = [0, 1, 2, 3]  # TODO: Active registers

# Variables
# NOTE: If the Rabi rates are different, you have to calibrate!
config_vars = ps.backend_simulation_vars(backend, rabi=False, units=units)

# Carrier frequencies of each control line
carriers = ps.backend_carriers(backend, config_vars)

config_vars

{'delta0': -0.3433479504893883,
 'delta1': -0.34621164922105846,
 'delta2': -0.3436602870914885,
 'delta3': -0.34355236125476857,
 'delta4': -0.3421076389828771,
 'jq0q1': 0.0018852610005410154,
 'jq1q2': 0.0019047408446458852,
 'jq2q3': 0.00197385774088835,
 'jq3q4': 0.0019396070264683515,
 'omegad0': 0.15915494309189535,
 'omegad1': 0.15915494309189535,
 'omegad2': 0.15915494309189535,
 'omegad3': 0.15915494309189535,
 'omegad4': 0.15915494309189535,
 'wq0': 4.962770879920025,
 'wq1': 4.838412258764765,
 'wq2': 5.036989248286842,
 'wq3': 4.95130021221037,
 'wq4': 5.066350584469811,
 't1q0': 109187.19287058488,
 't1q1': 57535.35189181149,
 't1q2': 183441.97711073843,
 't1q3': 109616.57783040682,
 't1q4': 102477.38825319844,
 't2q0': 5077.2297500997165,
 't2q1': 61650.15600725496,
 't2q2': 25123.784823624348,
 't2q3': 57120.186456626994,
 't2q4': 37229.852617362085}

In [5]:
# Partially compile to get this circuit's gates
cr_model = functools.partial(
    ps.cross_resonance_model,
    registers=registers,
    backend=backend,
    variables=config_vars, 
    model_name="Toy",
    return_params=true
)


# Control model
control = 0
target = 1
H_drift, Hs_control, H_channel, params = cr_model((control, target))

ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
def plot_Hamiltonian(H):
    fig, axes = plt.subplots(1, 2)
    kwargs = {"vmin": -.25, "vmax": .25, "cmap": "RdBu"}
    ax = axes[0]
    ax.imshow(np.real(H), **kwargs)
    ax = axes[1]
    ax.imshow(np.imag(H), **kwargs)
    [ax.axis("off") for ax in axes]
    return fig, axes

In [ ]:
plot_Hamiltonian(Hs_control)